In [5]:
from transformers import pipeline 
import pandas as pd 

### Text classification

In [6]:
classifier =  pipeline('text-classification') 

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [7]:
text = """L’AVEU
Jordan Bardella et Éric Zemmour ont ce point commun : ils vendent la mèche que Marine Le Pen essaye tant bien que mal de cacher. 
Bardella et Zemmour disent en effet tout haut ce que tout(e) dirigeant(e) d’extrême-droite pense tout bas. 
Zemmour a laissé entendre que Dreyfus était coupable et affirmé que Pétain avait sauvé des juifs. 
Et Bardella vient d’expliquer que le multirécidiviste Le Pen (père) n’était évidemment pas antisémite. 
Patientons un peu et il nous racontera bientôt  que Darquier de Pellepoix était un bon catholique et qu’Hitler a construit des autoroutes."""

In [10]:
outputs = classifier(text) 
pd.DataFrame(outputs)

,label,score
0,NEGATIVE,0.97104


### Named Entity Recognition

In [11]:
ner_tagger = pipeline("ner",aggregation_strategy="simple")
outputs = ner_tagger(text)
pd.DataFrame(outputs)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


All PyTorch model weights were used when initializing TFBertForTokenClassification.

All the weights of TFBertForTokenClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForTokenClassification for predictions without further training.


,entity_group,score,word,start,end
0,ORG,0.929315,L ’ AVEU,0,6
1,PER,0.988020,Jordan Bardella,7,22
2,PER,0.991996,Éric Zemmour,26,38
3,PER,0.975669,Marine Le Pen,86,99
4,PER,0.984812,Bardella,137,145
5,PER,0.967413,Zemmour,149,156
6,PER,0.974006,Zemmour,245,252
7,PER,0.951566,Dreyfus,275,282
8,PER,0.991126,Pétain,313,319
9,PER,0.866439,##t Bardella,345,355
